In [7]:
!ls

AUCell experiments.ipynb
Convert motif2TF extractions.ipynb
GSE60361_C1-3005-Expression.txt
Mouse TF gene identifiers to MGI symbols.ipynb
Profiling of recovery phase.ipynb
Test run of arboretum.ipynb
Test.ipynb
Whole-genome ranking - Benchmarking of storage solutions.ipynb
conv_B1DE4DB27D351516106646392.txt
dask-worker-space
data
hg19-500bp-upstream-10species.feather
hg19-500bp-upstream-7species.feather
hg19-tss-centered-10kb-10species.feather
hg19-tss-centered-10kb-7species.feather
hg19-tss-centered-5kb-10species.feather
hg19-tss-centered-5kb-7species.feather
hg19_direct_motifAnnotation.RData
hg19_inferred_motifAnnotation.RData
mm_tfs.txt
mydask.png
profile.html
pyscenic.ipynb
results.xlsx
tfs.txt


In [5]:
import pandas as pd
import numpy as np

In [17]:
ex_matrix = pd.read_csv('GSE60361_C1-3005-Expression.txt', sep='\t', header=0, index_col=0)

In [22]:
ex_matrix.head(25)

,1772071015_C02,1772071017_G12,1772071017_A05,1772071014_B06,1772067065_H06,1772071017_E02,1772067065_B07,1772067060_B09,1772071014_E04,1772071015_D04,...,1772066110_D12,1772071017_A07,1772063071_G10,1772058148_C03,1772063061_D09,1772067059_B04,1772066097_D04,1772063068_D01,1772066098_A12,1772058148_F03
cell_id,,,,,,,,,,,,,,,,,,,,,
Tspan12,0,0,0,3,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Tshz1,3,1,0,2,2,2,2,1,0,2,...,0,0,0,0,0,0,0,0,0,1
Fnbp1l,3,1,6,4,1,2,1,0,5,2,...,0,0,0,0,0,0,0,0,0,0
Adamts15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cldn12,1,1,1,0,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0
Rxfp1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,2,0,0,0,0,0,0
2310042E22Rik,0,0,2,3,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
Sema3c,11,0,25,1,10,0,7,0,8,28,...,0,0,1,0,0,0,0,0,0,0
Jam2,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1


In [21]:
ex_matrix.rank(axis=0, ascending=False)

,1772071015_C02,1772071017_G12,1772071017_A05,1772071014_B06,1772067065_H06,1772071017_E02,1772067065_B07,1772067060_B09,1772071014_E04,1772071015_D04,...,1772066110_D12,1772071017_A07,1772063071_G10,1772058148_C03,1772063061_D09,1772067059_B04,1772066097_D04,1772063068_D01,1772066098_A12,1772058148_F03
cell_id,,,,,,,,,,,,,,,,,,,,,
Tspan12,12410.5,12329.0,13000.5,2558.5,12337.0,12690.5,2615.5,12853.5,12616.0,12738.0,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,1312.0
Tshz1,1874.5,3856.0,13000.5,3469.5,2733.5,2984.5,3617.0,4520.5,12616.0,2961.5,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,1312.0
Fnbp1l,1874.5,3856.0,1243.0,1949.0,3949.0,2984.5,5227.5,12853.5,1168.5,2961.5,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
Adamts15,12410.5,12329.0,13000.5,12898.5,12337.0,12690.5,13108.5,12853.5,12616.0,12738.0,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
Cldn12,4003.0,3856.0,5095.0,12898.5,12337.0,12690.5,13108.5,12853.5,2888.5,2097.5,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
Rxfp1,12410.5,12329.0,13000.5,12898.5,12337.0,4482.0,13108.5,12853.5,4339.5,12738.0,...,11214.0,10540.0,11059.5,961.0,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
2310042E22Rik,12410.5,12329.0,3581.5,2558.5,12337.0,12690.5,13108.5,12853.5,12616.0,12738.0,...,11214.0,10540.0,366.0,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
Sema3c,296.0,12329.0,147.0,4917.0,415.5,12690.5,980.0,12853.5,643.5,90.0,...,11214.0,10540.0,1576.0,11437.5,10944.0,11097.0,10700.5,10975.0,10733.0,10874.0
Jam2,4003.0,12329.0,5095.0,12898.5,12337.0,4482.0,13108.5,12853.5,4339.5,12738.0,...,11214.0,10540.0,11059.5,11437.5,10944.0,11097.0,944.5,10975.0,960.5,1312.0


In [25]:
ex_matrix.memory_usage().sum()

480286656